# Trading Strategy for Finance using LSTMs

#### Model Construction

Now, we set up the TensorFlow compute graph. The deep neural network that is used in this code is comprised of a LSTM cell that runs over 10 time steps, a fully connected layers (FCL), and also drop-out layers to prevent overfitting. Calculating the number of time steps for a recurrent neural network is not a trivial task. It is actually another hyperparameter that needs to be searched. The network is depicted in the following figure.

<img src="imgs/dnn.jpg" width="700" height="700">

        Figure 3: Structure of the LSTM based deep neural network
                                
Below is the code to build the deep neural network depicted in Figure 3;

In [ ]:
def weight_variable(shape): 
    initial = tf.truncated_normal(shape, stddev=0.3)
    return tf.Variable(initial) 
    
def bias_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.3)
    return tf.Variable(initial) 

n_time_steps = 10
def getDNN (x, LSTMCellSize, keep_prob):
    with tf.name_scope('model'):
        with tf.name_scope('RNN'):
            # We will add two dropout layers and LSTM cells with the number of units as LSTMCellSize.
            cell = rnn.DropoutWrapper(rnn.BasicLSTMCell(LSTMCellSize, forget_bias=2, activation=tf.nn.tanh), output_keep_prob=keep_prob)
            # We use the cell to create RNN.
            # Note that outputs is not a tensor, it is a list with one element which is numpy array. 
            outputs, states = tf.nn.dynamic_rnn(cell, x, dtype=tf.float32) 
            outputs_shape = outputs.get_shape().as_list()
                
        # hidden layer with sigmoid activation
        with tf.name_scope('W_fc1'):
            W_fc1 = weight_variable([LSTMCellSize, 1])
        with tf.name_scope('b_fc1'):
            b_fc1 = bias_variable([1])
        with tf.name_scope('pred'):
            pred = tf.matmul(outputs[:,-1,:], W_fc1) + b_fc1

        return pred
    


In [ ]:
# The column names that will be included in the featureset are added into colList.
# colList will be used throughout the lab.
colList=[]                  
for thisColumn in df.columns: 
    if thisColumn not in ('id', 'timestamp', 'y', 'CntNs', 'y_lagged'): 
        colList.append(thisColumn)
colList.append('y_lagged')

#if you do not reset the default graph you will need to restart the kernel
#every time this notebook is run
tf.reset_default_graph()

# Network Parameters 
# Number of units in the LSTM cell.
n_LSTMCell = len(colList)

# Placeholder for the input and the keep probability for the dropout layers
with tf.name_scope('input'):
    x= tf.placeholder(tf.float32, shape=[None, n_time_steps, len(colList)])
with tf.name_scope('keep_prob'):
    keep_prob = tf.placeholder(tf.float32)

# At the input, we create 2-layer LSTM cell (with dropout layers)
print('Building tensorflow graph')

# Graph construction for the LSTM based deep neural network. 
# Structure of the network is depicted in the above figure.
# Please see the dnn.py to see the code of the network.
pred = getDNN (x, n_LSTMCell, keep_prob)

#### Training and Testing

We split the data into two pieces in time to have a training and testing set. In order to have enough sample for each id, the cut-off timestamp for the training set was defined in "insampleCutoffTimestamp" variable as 1650. Figure 4 shows how an instrument is split in time to create training and testing set. While training the model, the training set for each instrument will be fed separately to learn the time patterns in the data.

<img src="imgs/data_split.jpg" width="600" height="600">

                    Figure 4: Training and Testing Dataset
    
In the Kaggle challenge, the metric to evaluate the prediction accuracy was given as Pearson correlation. In statistics, [pearson correlation](https://en.wikipedia.org/wiki/Pearson_correlation_coefficient) is a measure of the linear correlation between two variables X and Y. It has a value between +1 and −1, where 1 is total positive linear correlation, 0 is no linear correlation, and −1 is total negative linear correlation. It is widely used in the sciences. It was developed by Karl Pearson from a related idea introduced by Francis Galton in the 1880s.

Depending on the frequency of the financial data, Pearson correlation (R) can be very small. In finance, given the high ratio of signal-to-noise, even a small R can deliver meaningful value. Please note that the algorithm that won the challenge had only 0.038 R.

The following cell includes the code for creating training and testing set, and calculating Pearson correlation.

In [ ]:
# Placeholder for the output (label)
with tf.name_scope('label'):
    y = tf.placeholder(tf.float32, shape=[None, 1]) 
# Placeholder to be able to split the data into training and test set while training the network.
inSampleCutoff = tf.placeholder(tf.int32, shape = ())

# this is important - we only want to train on the in-sample set of rows using TensorFlow
y_inSample = y[0:inSampleCutoff]
pred_inSample = pred[0:inSampleCutoff]

# also extract out of sample predictions and actual values,
# we'll use them for evaluation while training the model.
y_outOfSample = y[inSampleCutoff:]
pred_outOfSample = pred[inSampleCutoff:]

with tf.name_scope('stats'):
    # Pearson correlation to evaluate the model
    covariance = tf.reduce_sum(tf.matmul(tf.transpose(tf.subtract(pred_inSample, tf.reduce_mean(pred_inSample))),tf.subtract(y_inSample, tf.reduce_mean(y_inSample))))
    var_pred = tf.reduce_sum(tf.square(tf.subtract(pred_inSample, tf.reduce_mean(pred_inSample))))
    var_y = tf.reduce_sum(tf.square(tf.subtract(y_inSample, tf.reduce_mean(y_inSample))))
    pearson_corr = covariance / tf.sqrt(var_pred * var_y) 

tf.summary.scalar("pearson_corr", pearson_corr)

In most of the traditional machine learning and deep learning methods, it is assumed that the feature set and predicted value have zero mean and unit variance gaussian distribution. Empirical studies show that the financial data such as asset returns is often not compatible with this assumption. That is why we normalize the "y" variable by subtracting its mean and dividing the result by the standard deviation in the following cell. As an exercise, you can also normalize the features and see if you improve the accuracy.

In [ ]:
# Training dataset is also created here. We included the code to split the data in the above cell. 
# The difference is that the above code will be used in the training by the TensorFlow.
# This code will not be used by TensorFlow and creates the testing dataset whenever it is executed.
dfInSample = df[df.timestamp <  insampleCutoffTimestamp]
# create a reference dataframe (that only depends on in-sample data)
# that gives us standard deviation and mean information on per-id basis
# we'll use it later for variance stabilization
meanStdById = dfInSample.groupby(['id']).agg( {'y':['mean', 'std']})

We are ready to launch the graph for training the model and see intermediate diagnostics results and the final result. We defined the important hyperparameters including the epoch, training batch size and learning rate at the top of the cell. Initially, the epoch is set to 1 because it takes 15-20 minutes to complete the training with 10 epochs even though we are using GPUs. In order to speed up the training in the lab environment, we provided pre-trained networks with 10 epochs and 20 epochs. An adaptive learning rate starting from 0.002 with exponential decay is used for the training from scratch. Learning rate should be set to 0.00058 and 0.00061 for using pre-trained models with 10 and 15 epochs respectively.

In [ ]:
# Training parameters
display_step = 100 
epoch = 1
pre_trained_model = 'tasks/task2/task/SavedModels/model_epoch_10.ckpt'
mini_batch_limit = 1300

# set up adaptive learning rate:
globalStep = tf.placeholder(tf.float32)
# Ratio of globalStep / totalDecaySteps is designed to indicate how far we've progressed in training.
# the ratio is 0 at the beginning of training and is 1 at the end.
# adaptiveLearningRate will thus change from the starting learningRate to learningRate * decay_rate
# in order to simplify the code, we are fixing the total number of decay steps at 1 and pass globalStep
# as a fraction that starts with 0 and tends to 1.
# Learning rate should be set to 0.002 if you are training from scratch.
# Learning rate should be set to 0.00058 if you are using the pre-trained network with 10 epochs.
# Learning rate should be set to 0.00061 if you are using the pre-trained network with 15 epochs.
adaptiveLearningRate = tf.train.exponential_decay(
  0.00058,       # Start with this learning rate
  globalStep,  # globalStep / totalDecaySteps shows how far we've progressed in training
  1,           # totalDecaySteps
  0.3)         # decay_rate, the factor by which the starting learning rate will be 
               # multiplied when the training is finished
    
# Define loss and optimizer
# Note the loss only involves in-sample rows
# Regularization is added in the loss function to avoid over-fitting
rnn_variables = lstm_variables = [v for v in tf.trainable_variables()
                    if v.name.startswith('rnn')]

with tf.name_scope('loss'):
    loss = tf.nn.l2_loss(tf.subtract(y_inSample,pred_inSample)) + tf.contrib.layers.apply_regularization(tf.contrib.layers.l2_regularizer(scale=0.0001), tf.trainable_variables())

tf.summary.scalar("loss", loss)
optimizer = tf.train.AdamOptimizer(learning_rate=adaptiveLearningRate).minimize (loss) 

# Getting unique ids to train the network per id basis.
ids = df.id.unique()
ids.sort()

summary_op = tf.summary.merge_all()

# initialize the variables 
init = tf.global_variables_initializer()

totalActual = []
totalPredicted = []
import random
# Launch the graph 
# Implement Cross Validation, but in a vay that preserves temporal structure for id's 
with tf.Session() as sess:  
    # Global variables are initialized
    sess.run(init) 
    
    # Restore latest checkpoint
    model_saver = tf.train.Saver()
    model_saver.restore(sess, pre_trained_model)
    
    writer = tf.summary.FileWriter("tasks/tensorboard/logs", graph=tf.get_default_graph())
    step = 50  
    writer_step = 1;
    for i in range(epoch):
        print('Epoch: ', i, '******************************')        
        actual = []
        predicted = []
        
        random.shuffle(ids)

        for thisId in ids:
            # Getting the data of the current id
            this_df = df[df.id == thisId].copy()
            this_df = this_df.sort_values(['id', 'timestamp'])
                        
            # we need to pass training set to the graph definition
            # optimization will only consider in training set
            inSampleSize, _ = this_df[this_df.timestamp < insampleCutoffTimestamp].shape
            totalRows, _ = this_df.shape
            
            batch_y = this_df.loc[:,'y'].values            
            batch_x = this_df[colList].values
                    
            if totalRows < n_time_steps:
                continue

            # Data is formated as a 3D tensor with the shape of (batch_size, n_time, n_feature) for LSTM
            # n_time_steps parameter determines how many steps that LSTM will unroll in time
            complete_x = np.zeros([totalRows-n_time_steps+1, n_time_steps, len(colList)])
            for n in range(n_time_steps):
                complete_x[:,n,:]=batch_x[n:totalRows-n_time_steps+n+1,:]
            
            batch_y = batch_y[n_time_steps-1:]
            inSampleSize -= n_time_steps - 1

            # variance stabilizing transform
            # some id's will not have in-sample rows, we cannot perform transform on those
            # furthermore, since there is not in-sample rows to train on, we must skip
            if inSampleSize < 10:
                continue
                
            # perform variance stabilization
            thisMean = meanStdById.loc[thisId][0]
            thisStd = meanStdById.loc[thisId][1]
            batch_y = (batch_y - thisMean) / thisStd
            
            batch_y = batch_y.reshape(-1,1)
            minibatchSize, _ = batch_y.shape

            # we want to make sure that RNN reaches steady state
            if minibatchSize < mini_batch_limit: 
                continue 
            
            # Run optimization 
            # note: keep_prob is set to 0.5 for training only!
            _, currentRate = sess.run([optimizer, adaptiveLearningRate], feed_dict={x: complete_x, y: batch_y, keep_prob:0.5, inSampleCutoff:inSampleSize, globalStep:i/epoch})

            # Obtain out of sample target variable and our prediction
            y_oos, pred_oos = sess.run([y_outOfSample, pred_outOfSample], feed_dict={x: complete_x, y: batch_y, keep_prob:1.0, inSampleCutoff:inSampleSize}) 
            
            # flatten the returned lists
            y_oos = [y for x in y_oos for y in x]
            pred_oos = [y for x in pred_oos for y in x]
            
            #reverse transform before recording the results
            if inSampleSize:            
                y_oos = [ (t*thisStd + thisMean) for t in y_oos]
                pred_oos = [ (t*thisStd + thisMean) for t in pred_oos]
            
            # record the results
            actual.extend(y_oos)
            predicted.extend(pred_oos)
                       
            totalActual.extend(y_oos)
            totalPredicted.extend(pred_oos)
            
            # Once every display_step show some diagnostics - the loss function, in-sample correlation, etc.
            if step % display_step == 0: 
                # Calculate batch accuracy 
                # Calculate batch loss 
                correl, lossResult, summary = sess.run([pearson_corr, loss, summary_op], feed_dict={x: complete_x, y: batch_y, keep_prob:1.0, inSampleCutoff:inSampleSize})
                
                writer.add_summary(summary, writer_step)
                writer_step += 1
                # corrcoef sometimes fails to compute correlation for a perfectly valid reason (e.g. stdev(pred_oos) is 0)
                # it sets the result to nan, but also gives an annoying warning
                # the following suppresses the warning
                with warnings.catch_warnings():
                    warnings.simplefilter("ignore")
                    correl_oos = np.corrcoef(y_oos, pred_oos)[0,1]
                    
                print('LR: %s - Iter %s, minibatch loss = %s, minibatch corr = %s, oos %s (%s/%s)' % (currentRate, step, lossResult, correl, correl_oos, inSampleSize, totalRows))
                
            step += 1 
       
        print('Optimization Finished!') 
        print('Correl: ', np.corrcoef(actual, predicted)[0,1])

It takes 3-5 minutes to complete the training with 1 epochs. We also provided TensorBoard to review the model architecture, loss and correlation variables. TensorBoard is a suite of web applications for inspecting and understanding your TensorFlow runs and graphs. 

### Click [here](/tensorboard/) to start TensorBoard.

You should get a correlation value around R = 0.04. Note that the correlation tends to increase with each epoch (but not always). 

Let's use the pre-trained model with 15 epochs by setting the pre_trained_model variable as **pre_trained_model = '/tasks/task2/task/SavedModels/model_epoch_15.ckpt'** in above cell, lower the starting Learning Rate to **0.00061** and re-run everything using Kernel->Restart & Run All.

What is the correlation that you get this time? Was it improved?

Since training takes significant amount of time, we recommend you train the model with 20 epochs and check the correlation after this lab in your environment. You should get a correlation value around R = 0.05.
